# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [ ]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [108]:
test = pd.read_csv('flights_test.csv')

In [109]:
df = pd.read_csv('flights_merged.csv', index_col=0)

In [110]:
df = df.dropna(axis=0, subset=['crs_elapsed_time'])

In [111]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,passengers,capacity,total_gallons,dep_delay_y,nas_delay_y,security_delay_y,late_aircraft_delay_y,weather_delay_y,carrier_delay_y
0,2019-01-01,AA,AA_CODESHARE,AA,5242,OH,N237PS,5242,11278,DCA,...,248.0,128.0,0.0,0.000000e+00,60.009434,8.179245,0.0,36.910377,3.018868,14.910377
1,2019-01-03,AA,AA_CODESHARE,AA,5242,OH,N254PS,5242,11278,DCA,...,248.0,128.0,0.0,0.000000e+00,60.009434,8.179245,0.0,36.910377,3.018868,14.910377
2,2019-01-14,AA,AA_CODESHARE,AA,5242,OH,N249PS,5242,11278,DCA,...,248.0,128.0,0.0,0.000000e+00,60.009434,8.179245,0.0,36.910377,3.018868,14.910377
3,2019-01-16,AA,AA_CODESHARE,AA,5556,OH,N202PS,5556,11278,DCA,...,248.0,128.0,0.0,0.000000e+00,60.009434,8.179245,0.0,36.910377,3.018868,14.910377
4,2019-01-17,AA,AA_CODESHARE,AA,5556,OH,N209PS,5556,11278,DCA,...,248.0,128.0,0.0,0.000000e+00,60.009434,8.179245,0.0,36.910377,3.018868,14.910377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807073,2019-12-14,F9,F9,F9,1531,F9,N343FR,1531,11618,EWR,...,746.0,2732.0,244.0,2.353042e+07,26.166667,14.000000,0.0,16.000000,0.000000,7.166667
807074,2019-12-15,F9,F9,F9,1531,F9,N227FR,1531,11618,EWR,...,746.0,2732.0,244.0,2.353042e+07,26.166667,14.000000,0.0,16.000000,0.000000,7.166667
807075,2019-12-17,F9,F9,F9,1531,F9,N228FR,1531,11618,EWR,...,746.0,2732.0,244.0,2.353042e+07,26.166667,14.000000,0.0,16.000000,0.000000,7.166667
807076,2019-12-18,F9,F9,F9,1531,F9,N346FR,1531,11618,EWR,...,746.0,2732.0,244.0,2.353042e+07,26.166667,14.000000,0.0,16.000000,0.000000,7.166667


In [112]:
y= df['arr_delay']

In [113]:
X = df

In [114]:
X = X.drop(columns=['arr_delay'])

In [88]:
#X.fl_date = X.fl_date.apply(pd.to_datetime)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [ ]:
X = X[['origin_airport_id', 'dest_airport_id', 'air_time', 'distance', 'passengers', 'crs_elapsed_time','capacity', 'total_gallons', 'dep_delay_y',
       'nas_delay_y', 'security_delay_y', 'late_aircraft_delay_y',
       'weather_delay_y', 'carrier_delay_y']]
#dropped flight date :( 

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [92]:
#data_dmatrix = xgb.DMatrix(data=X,label=y, enable_categorical=True)

In [93]:
#objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 7, alpha = 0.1,

In [116]:
scaler = StandardScaler()

In [117]:
X = pd.DataFrame(scaler.fit_transform(X))

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8)


In [119]:
lr = LinearRegression()

In [120]:
lr.fit(X_train, y_train) 

LinearRegression()

In [121]:
lr.score(X_train, y_train)

0.03795838486980141

In [122]:
lr.score(X_test, y_test)

0.03657385654892786

### Random Forest

RandomForestRegressor()

In [ ]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100)
 

In [ ]:
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

In [125]:
y_pred = regressor.predict(X_test)

In [ ]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

0.6354390805496346

-0.2055041190161382

In [135]:
params = {'n_estimators' : [100, 150, 200, 300], 'max_depth': [3, 10, 20, 30]}

In [136]:
grid = GridSearchCV(regressor, params)

In [ ]:
grid_result = grid.fit(X_train, y_train)

In [ ]:
print(grid_result.best_score_)
print( grid_result.best_params_['n_estimators'])
print( grid_result.best_params_['max_depth'])

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.